# Machine Learning - Linear classification

In [ ]:
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt

## Datasets

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X_iris, y_iris = iris.data, iris.target

## Preprocessing
- Get dataset with only the first two attributes
- Split the dataset into a training and testing set. Test set will be the 25% taken randomly

In [ ]:
from sklearn.model_selection import train_test_split

X, y = X_iris[:,0:2], y_iris
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

## Standarize the features

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Draw the train set

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

colors = ['red', 'greenyellow', 'blue']
for i in range(len(colors)):
    x_scatter = X_train[:,0][y_train == i]
    y_scatter = X_train[:,1][y_train == i]
    plt.scatter(x_scatter, y_scatter, c=colors[i])
    
plt.legend(iris.target_names)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width');

## Stochastic Gradient Descend

In [ ]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(max_iter=1000, tol=1e-3)
classifier.fit(X_train, y_train)

print(classifier.coef_)
print(classifier.intercept_)

## Draw the hyperplanes

In [ ]:
x_min, x_max = X_train[:, 0].min() - 0.5, X_train[:, 0].max() + 0.5
y_min, y_max = X_train[:, 1].min() -0.5, X_train[:, 1].max() + 0.5
xs = np.arange(x_min, x_max, 0.5)

fig, axes = plt.subplots(1, 3)
fig.set_size_inches(12, 12)
for i in range(0, 3):
    axes[i].set_aspect('equal')
    axes[i].set_title('Class ' + str(i) + ' versus the rest')
    axes[i].set_xlabel('Sepal length')
    axes[i].set_ylabel('Sepal width')
    axes[i].set_xlim(x_min, x_max)
    axes[i].set_ylim(y_min, y_max)
    plt.sca(axes[i])
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=plt.cm.prism)
    ys = -(classifier.intercept_[i] + xs*classifier.coef_[i, 0])/classifier.coef_[i, 1]
    plt.plot(xs, ys)

## A prediction

In [ ]:
print(classifier.predict(scaler.transform([[4.7, 3.1]])))
print(classifier.decision_function(scaler.transform([[4.7, 3.1]])))

## Evaluating the results

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = classifier.predict(X_train)
print(accuracy_score(y_train, y_pred))
print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))

In [ ]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


## Cross Validation

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline

# Create a composite estimator made by a pipeline of the standaritation and the linear model
classifier = Pipeline([('scaler', StandardScaler()),
    ('linear_model', SGDClassifier(max_iter=1000, tol=1e-3))])

# By default the score used is the one returned by score method of the estimator (accuracy)
scores = cross_val_score(classifier, iris.data, iris.target, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))